In [2]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
import plotly.graph_objects as go
from statsmodels.stats.outliers_influence import OLSInfluence

In [3]:
def load_csv_to_dataframe(file_path):
    """Load a CSV file into a Pandas DataFrame."""
    try:
        df = pd.read_csv(file_path, encoding='latin1')
        print(f"Successfully loaded data from {file_path}")
        print(df.head())
        return df
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except pd.errors.EmptyDataError:
        print(f"Error: The file {file_path} is empty.")
    except pd.errors.ParserError:
        print(f"Error: The file {file_path} could not be parsed.")
    except UnicodeDecodeError as e:
        print(f"Encoding error while reading the file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [4]:
if __name__ == "__main__":
    file_path = 'sales_data_sample.csv'
    df = load_csv_to_dataframe(file_path)

Successfully loaded data from sales_data_sample.csv
   ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES  \
0        10107               30      95.70                2  2871.00   
1        10121               34      81.35                5  2765.90   
2        10134               41      94.74                2  3884.34   
3        10145               45      83.26                6  3746.70   
4        10159               49     100.00               14  5205.27   

         ORDERDATE   STATUS  QTR_ID  MONTH_ID  YEAR_ID  ...  \
0   2/24/2003 0:00  Shipped       1         2     2003  ...   
1    5/7/2003 0:00  Shipped       2         5     2003  ...   
2    7/1/2003 0:00  Shipped       3         7     2003  ...   
3   8/25/2003 0:00  Shipped       3         8     2003  ...   
4  10/10/2003 0:00  Shipped       4        10     2003  ...   

                    ADDRESSLINE1  ADDRESSLINE2           CITY STATE  \
0        897 Long Airport Avenue           NaN            NYC    

In [5]:
# IQR method AFTER DFFITS
q1 = df['SALES'].quantile(0.25)
q3 = df['SALES'].quantile(0.75)
iqr = q3 - q1   
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers_iqr = df[(df['SALES'] < lower_bound) | (df['SALES'] > upper_bound)]
print(f"Outliers based on IQR:\n", outliers_iqr)

Outliers based on IQR:
       ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER     SALES  \
30          10150               45      100.0                8  10993.50   
32          10174               34      100.0                4   8014.82   
35          10206               47      100.0                6   9064.89   
41          10280               34      100.0                2   8014.82   
43          10304               47      100.0                6  10172.70   
...           ...              ...        ...              ...       ...   
2241        10325               38      100.0                3   8844.12   
2295        10324               48      100.0                4   8209.44   
2430        10395               45      100.0                3   8977.05   
2505        10388               46      100.0                2  10066.60   
2634        10336               46      100.0                2   9558.80   

            ORDERDATE   STATUS  QTR_ID  MONTH_ID  YEAR_ID  ... 

In [6]:
# Plotting IQR
fig_iqr = px.box(df, y='SALES', points='all', title='Box Plot of Sales Data with IQR Outliers')
fig_iqr.add_scatter(y=[lower_bound, lower_bound], mode='lines', name='Lower Bound', line=dict(color='red', dash='dash'))
fig_iqr.add_scatter(y=[upper_bound, upper_bound], mode='lines', name='Upper Bound', line=dict(color='green', dash='dash'))
fig_iqr.show()

In [7]:
# Plotting IQR with outliers highlighted
fig_iqr_outliers = px.box(df, y='SALES', points='all', title='Box Plot of Sales Data with IQR Outliers Highlighted')
fig_iqr_outliers.add_scatter(y=[lower_bound, lower_bound], mode='lines', name='Lower Bound', line=dict(color='red', dash='dash'))
fig_iqr_outliers.add_scatter(y=[upper_bound, upper_bound], mode='lines', name='Upper Bound', line=dict(color='green', dash='dash'))
fig_iqr_outliers.add_scatter(y=df['SALES'][df['SALES'] < lower_bound], mode='markers', name='Lower Outliers', marker=dict(color='orange', size=10))
fig_iqr_outliers.add_scatter(y=df['SALES'][df['SALES'] > upper_bound], mode='markers', name='Upper Outliers', marker=dict(color='purple', size=10))
fig_iqr_outliers.show()

In [8]:
# Plotting IQR with outliers highlighted and lower and upper bounds
fig_iqr_outliers_bounds = px.box(df, y='SALES', points='all', title='Box Plot of Sales Data with IQR Outliers Highlighted and Bounds')
fig_iqr_outliers_bounds.add_scatter(y=[lower_bound, lower_bound], mode='lines', name='Lower Bound', line=dict(color='red', dash='dash'))
fig_iqr_outliers_bounds.add_scatter(y=[upper_bound, upper_bound], mode='lines', name='Upper Bound', line=dict(color='green', dash='dash'))
fig_iqr_outliers_bounds.add_scatter(y=df['SALES'][df['SALES'] < lower_bound], mode='markers', name='Lower Outliers', marker=dict(color='orange', size=10))
fig_iqr_outliers_bounds.add_scatter(y=df['SALES'][df['SALES'] > upper_bound], mode='markers', name='Upper Outliers', marker=dict(color='purple', size=10))
fig_iqr_outliers_bounds.show()

In [10]:
# Apllying DFFITS method after filtered outliers from IQR
# Fit the model again after removing IQR outliers
df_noOutliers = df[(df['SALES'] >= lower_bound) & (df['SALES'] <= upper_bound)]
model_noOutliers = sm.OLS(df_noOutliers['SALES'], sm.add_constant(df_noOutliers[['PRICEEACH', 'QUANTITYORDERED']])).fit()
# Get the influence measures
influence_noOutliers = OLSInfluence(model_noOutliers)
# Get the DFFITS values
dffits = influence_noOutliers.dffits[0]
# Identify the outliers using absolute values
outliers_dffits_noOutliers = abs(dffits) > 2 * dffits.std()
print(f"Outliers based on DFFITS after IQR filtering:\n",df_noOutliers[outliers_dffits_noOutliers])

Outliers based on DFFITS after IQR filtering:
       ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES  \
20          10341               41     100.00                9  7737.93   
22          10375               21      34.91               12   733.11   
26          10103               26     100.00               11  5404.62   
27          10112               29     100.00                1  7209.11   
28          10126               38     100.00               11  7329.06   
...           ...              ...        ...              ...      ...   
2586        10401               85      88.75               10  7543.75   
2656        10318               50     100.00                8  7119.00   
2662        10388               50     100.00                3  7154.50   
2689        10401               77      92.00                9  7084.00   
2772        10131               21      41.71                7   875.91   

            ORDERDATE   STATUS  QTR_ID  MONTH_ID  YE

In [11]:
# Plotting DFFITS after IQR filtering
fig_dffits_noOutliers = px.scatter(x=dffits, y=model_noOutliers.resid, title='DFFITS vs Residuals after IQR Filtering', labels={'x': 'DFFITS', 'y': 'Residuals'})
fig_dffits_noOutliers.add_scatter(x=dffits, y=[0]*len(dffits), mode='lines', name='Zero Line', line=dict(color='red', dash='dash'))
fig_dffits_noOutliers.add_scatter(x=dffits[outliers_dffits_noOutliers], y=model_noOutliers.resid[outliers_dffits_noOutliers], mode='markers', name='Outliers', marker=dict(color='orange', size=10))
fig_dffits_noOutliers.show()

In [ ]:
lower_bound_dffits_noOutliers = dffits.mean() - 2 * dffits.std()
upper_bound_dffits_noOutliers = dffits.mean() + 2 * dffits.std()
fig_dffits_bounds_noOutliers = px.scatter(x=dffits, y=model_noOutliers.resid, title='DFFITS vs Residuals with Bounds after IQR Filtering', labels={'x': 'DFFITS', 'y': 'Residuals'})
fig_dffits_bounds_noOutliers.add_scatter(x=dffits, y=[0]*len(dffits), mode='lines', name='Zero Line', line=dict(color='red', dash='dash'))
fig_dffits_bounds_noOutliers.add_scatter(x=dffits, y=[lower_bound_dffits_noOutliers]*len(dffits), mode='lines', name='Lower Bound', line=dict(color='blue', dash='dash'))
fig_dffits_bounds_noOutliers.add_scatter(x=dffits, y=[upper_bound_dffits_noOutliers]*len(dffits), mode='lines', name='Upper Bound', line=dict(color='green', dash='dash'))
fig_dffits_bounds_noOutliers.add_scatter(x=dffits[outliers_dffits_noOutliers], y=model_noOutliers.resid[outliers_dffits_noOutliers], mode='markers', name='Outliers', marker=dict(color='orange', size=10))
fig_dffits_bounds_noOutliers.show()

: 